# Import/Class/Function

In [19]:
%matplotlib
import sys
sys.path.append('..\Instrument Driver')
sys.path.append('..\Helper') 

import os
import re
import json
import pyvisa
import time
import addict  # allow dot notation for dictionary
import matplotlib.pyplot as plt   
from scipy.optimize import curve_fit
import numpy as np
from plotter import QuadPlotter  
import logging

logging.basicConfig(level=logging.INFO) # set logging level to be INFO.

def create_full_file_name_and_path(exp_params):
    """
    Create a full file name and full file path, update exp_params. 
    """
    exp_params.full_file_name = get_full_file_name(exp_params.base_path
                                                   , exp_params.device_name
                                                   , exp_params.exp_name)
    
    exp_params.full_file_path = os.path.join(exp_params.base_path
                                             , exp_params.full_file_name)
   
    return exp_params

def get_full_file_name(base_path, device_name, exp_name):
    """ 
    Create a full file name with file extension 'dat'.
    File name structure = ID number_Sample ID_Exp name.json. 
    ex) 1_NbTi_VI.json
    """
    full_file_name = (get_file_ID_number(base_path) 
                      + '_' 
                      + device_name 
                      + '_' 
                      + exp_name 
                      + '.json')

    return full_file_name

def get_file_ID_number(base_path):
    """ Find ID number used to create a full file name. 
        Under base path, search for all ID numbers of files and decide
        what ID to be used next.
        Args: 
                base_path: base path
        Return: next ID number in str to be used
        """
    ID_numbers = []
    for (root,dirs,files) in os.walk(base_path, topdown=True):
        
        for file in files:
            matchobj = re.match(r'^\d+', file)
            if matchobj:
                ID_numbers.append(int(matchobj.group()))    
    if ID_numbers:
        return str(np.max(ID_numbers) + 1)
    else:
        return str(1)

def save_data(exp_params):

    with open(exp_params.full_file_path, 'w') as f:
        json.dump(exp_params, f, indent=4, sort_keys=True)
        logging.info(f'{exp_params.full_log_file_path} created.')

Using matplotlib backend: Qt5Agg


# Create instrument driver instances and set common parameters

In [21]:
import SIM, LSCI332S, HittiteHMC, SynthHD, KeysightP9372A

pna = KeysightP9372A.KeysightP9372A('PXI10::0-0.0::INSTR') # pna.connect()
flux1 = SIM.SIM(18, channel = 1)
flux2 = SIM.SIM(18, channel = 2)
spec = HittiteHMC.HittiteHMC('USB',4)

# set common parameters throughout various measurements
exp_params_common = addict.Dict({})
exp_params_common.base_path = r'Z:\User\Jaseung\temp\temp'
exp_params_common.device_name = 'transmonQ1'
exp_params_common.VNA_fixed_att = 40

ModuleNotFoundError: No module named 'visa'

# Cav 1D

In [34]:
# set parameters
exp_params = Dict({ 'exp_name': 'cav'
                    ,'fixed_atten': 30
                    , 'freq_start': 1e6
                    , 'freq_end': 9e9
                    , 'power': -30
                    , 'num_points': 801
                    , 'avg': 2
                    , 'IF': 1e3                    
                    })
exp_params.update(exp_params_common)

def run_cav_1D(exp_params):
    
    # file
    exp_params = create_full_file_name_and_path(exp_params)

    # plot initialization
    exp_params.xlabel = 'Frequency (Hz)'
    plotter = QuadPlotter(plot_dim=1
                        , xlabel=exp_params.xlabel
                        , title=exp_params.full_file_name
                        , figsize=(12,10))

    # instrument initialization
    pna.setupMeas('S21')
    pna.setSweepType('linear')
    pnaNone.setFreqStartEnd(exp_params.freq_start, exp_params.freq_end)
    pna.numPoints(exp_params.num_points)
    pna.avgCount(exp_params.avg)    
    pna.setIF(exp_params.IF)
    pna.setPwr(exp_params.power)

    # start measurement
    freq, S21 = pna.getData(cplx=True) # S21 in linear scale

    plotter.update(xdata=freq, ydata=S21)

    # save data and exp parameters
    exp_params.xdata, exp_params.data = freq, S21
    save_data(exp_params)

    # Post measurement
    plotter.fig.savefig(exp_params.full_file_path.replace('json','png'))
    logging.info('Measurement complete!')
    
    return exp_params

data = run_cav_1D(exp_params)

# Cav vs Power 2D

In [6]:
# Set parameters 
exp_params = Dict({'exp_name': 'Cav_vs_P_2D'
                    , 'freq_start': 1e6
                    , 'freq_end': 9e9
                    , 'power': -30
                    , 'num_points': 801
                    , 'avg': 2
                    , 'IF_start': 500
                    , 'IF_end': 1e3
                    , 'power_start': -30
                    , 'power_end': -10
                    , 'power_step': 1
})
exp_params.update(exp_params_common)

def run_cav_vs_P_2D(exp_params):
    # file
    exp_params = create_full_file_name_and_path(exp_params)

    # plot initialization
    exp_params.xlabel = 'Power (dBm)'
    exp_params.ylabel = 'Frequency (Hz)'
    plotter = QuadPlotter(plot_dim=2
                        , xlabel=exp_params.xlabel
                        , ylabel=exp_params.ylabel
                        , title=exp_params.full_file_name
                        , figsize=(12,10))

    # initialize instruments
    pna.setupMeas('S21')
    pna.setSweepType('linear')
    pna.setFreqStartEnd(exp_params.freq_start, exp_params.freq_end)
    pna.avgCount(exp_params.avg)
    pna.numPoints(exp_params.num_points)

    # flux.setVoltage(exp_params.flux_init)

    # sweep
    powers = np.arange(exp_params.power_start, exp_params.power_end + exp_params.power_step, exp_params.power_step)
    IFBandwiths = np.linspace(exp_params.IF_start, exp_params.IF_end, len(powers)) # IFbandwidth scales up linearly

    data, power_list = np.array([]), np.array([])

    for pwr, IF in zip(powers, IFBandwiths):

        # configure
        pna.setIF(IF)
        pna.setPwr(pwr)
        pna.avgClear()

        freq, S21 = pna.getData(cplx=True) # S21 in linear scale

        if data.size > 0:
            data = np.column_stack((data, S21))
        else:
            data = np.reshape(S21,(len(S21),1))

        # plot update
        power_list = np.append(power_list, pwr)
        plotter.update(x_axis=power_list, y_axis=freq, data=data)

    # save data
    exp_params.xdata = power_list
    exp_params.ydata = freq
    exp_params.data = data
    save_data(exp_params)

    # Post measurement
    plotter.fig.savefig(exp_params.full_file_path.replace('json','png'))
    logging.info('Measurement complete!')
    
    return exp_params

data = run_cav_vs_P_2D(exp_params)

NameError: name 'base_path' is not defined

# Cav vs Flux 2D 

In [ ]:
# Set parameters 
exp_params = Dict({'exp_name': 'Cav_vs_flux_2D'
                    ,'freq_start': 1e6
                    , 'freq_end': 9e9
                    , 'power': -30
                    , 'num_points': 801
                    , 'avg': 2
                    , 'IF': 1e3
                    , 'flux_start': -1
                    , 'flux_end': 1
                    , 'flux_step': 1
                    })
exp_params.update(exp_params_common)

def run_cav_vs_flux_2D(exp_params):
    # file
    exp_params = create_full_file_name_and_path(exp_params)

    # plot initialization
    exp_params.xlabel = 'Flux (V)'
    exp_params.ylabel = 'Frequency (Hz)'
    plotter = QuadPlotter(plot_dim=2
                        , xlabel=exp_params.xlabel
                        , ylabel=exp_params.ylabel
                        , title=exp_params.full_file_name
                        , figsize=(12,10))

    # initialize instruments
    pna.setupMeas('S21')
    pna.setSweepType('linear')
    pna.setFreqStartEnd(exp_params.freq_start, exp_params.freq_end)
    pna.setPwr(exp_params.power)
    pna.setIF(exp_params.IF)
    pna.avgCount(exp_params.avg)
    pna.numPoints(exp_params.num_points)

    # sweep
    flux_sweep = np.arange(params.flux_start, params.flux_end + params.flux_step, params.flux_step)
    data, flux_list = np.array([]), np.array([])
    for flux in flux_sweep:

        # configure
        pna.avgClear()

        freq, S21 = pna.getData(cplx=True) # S21 in linear scale

        if data.size > 0:
            data = np.column_stack((data, S21))
        else:
            data = np.reshape(S21,(len(S21),1))

        # plot update
        flux_list = np.append(flux_list, flux)
        plotter.update(x_axis=flux_list, y_axis=freq, data=data)

    # save data
    exp_params.xdata = power_list
    exp_params.ydata = freq
    exp_params.data = data
    save_data(exp_params)

    # Post measurement
    plotter.fig.savefig(exp_params.full_file_path.replace('json','png'))
    logging.info('Measurement complete!')
    
    return exp_params

data = run_cav_vs_flux_2D(exp_params)

# 2D flux, fixed cavity

In [ ]:
# Set parameters 
exp_params = Dict({'exp_name': 'flux_2D'
                    ,'CW_freq': 6e9                  
                    , 'power': -30
                    , 'num_points': 801
                    , 'avg': 2
                    , 'IF': 1e3
                    , 'flux1_start': -1
                    , 'flux1_end': 1
                    , 'flux1_step': 1
                    , 'flux2_start': -1
                    , 'flux2_end': 1
                    , 'flux2_step': 1
                    })
exp_params.update(exp_params_common)

def run_flux_2D(exp_params):
        # file
    exp_params = create_full_file_name_and_path(exp_params)

    # plot initialization
    exp_params.xlabel = 'Flux1 (V)'
    exp_params.ylabel = 'Flux2 (V)'
    plotter = QuadPlotter(plot_dim=2
                        , xlabel=exp_params.xlabel
                        , ylabel=exp_params.ylabel
                        , title=exp_params.full_file_name
                        , figsize=(12,10))

    # initialize instruments
    pna.setupMeas('S21')
    pna.setSweepType('CW')
    pna.setCWFreq(exp_params.CW_freq)
    pna.setPwr(exp_params.power)
    pna.setIF(exp_params.IF)
    pna.avgCount(exp_params.avg)
    pna.numPoints(exp_params.num_points)

    # sweep
    flux1_sweep = np.arange(exp_params.flux1_start, exp_params.flux1_end + exp_params.flux1_step, exp_params.flux1_step)
    flux2_sweep = np.arange(exp_params.flux2_start, exp_params.flux2_end + exp_params.flux2_step, exp_params.flux2_step)
    data = np.array([])
    for flux1 in flux1_sweep:
        flux1.setVoltage(flux1)

        S21_list = np.array([])
        for flux2 in flux2_sweep:
            flux2.setVoltage(flux2)
            pna.avgClear()

             _, S21 = pna.getData(cplx=True) # S21 in linear scale
            S21_avg = np.mean(S21)
            S21_list = np.append(S21_list, S21_avg)        

        if data.size > 0:
            data = np.column_stack((data, S21_list))
        else:
            data = np.reshape(S21_list,(len(S21_list),1))

        # plot update
        plotter.update(x_axis=flux1_sweep, y_axis=flux2_sweep, data=data)

    # save data
    exp_params.xdata = flux1_sweep
    exp_params.ydata = flux2_sweep
    exp_params.data = data
    save_data(exp_params)

    # Post measurement
    plotter.fig.savefig(exp_params.full_file_path.replace('json','png'))
    logging.info('Measurement complete!')
    
    return exp_params

data = run_flux_2D(exp_params)

# Spec 1D

In [3]:
# Set parameters 
exp_params = Dict({'exp_name': 'spec_1D'
                    , 'CW_freq': 6e9
                    , 'power': -30                     
                    , 'num_points': 801
                    , 'avg': 2
                    , 'IF': 1e3
                    , 'spec_freq_start': 4e9
                    , 'spec_freq_start': 5e9
                    , 'spec_freq_start': 5e6
                    , 'spec_pwr' : -20
                    , 'spec_fixed_att' : 20
                    , 'flux': -0.2
                    })
exp_params.update(exp_params_common)

def run_spec_1D(exp_params):

    # file
    exp_params = create_full_file_name_and_path(exp_params)

    # plot initialization
    exp_params.xlabel = 'Spec Frequency (Hz)' 
    plotter = QuadPlotter(plot_dim=1
                        , xlabel=exp_params.xlabel                    
                        , title=exp_params.full_file_name
                        , figsize=(12,10))

    # initialize instruments
    pna.setupMeas('S21')
    pna.setSweepType('CW')
    pna.setCWFreq(exp_params.CW_freq)
    pna.setPwr(exp_params.power)
    pna.setIF(exp_params.IF)
    pna.avgCount(exp_params.avg)
    pna.numPoints(exp_params.num_points)
    
    spec.setPwr(exp_params.spec_pwr)
    
    flux.setVoltage(exp_params.flux)

    # sweep
    spec_sweep = np.arange(exp_params.spec_freq_start, exp_params.spec_freq_end + exp_params.spec_freq_step, exp_params.spec_freq_step) 
    data = np.array([])
    for spec in spec_sweep:

        # configure
        spec.setFreq(spec)
        pna.avgClear()

        _, S21 = pna.getData(cplx=True) # S21 in linear scale
        S21_avg = np.mean(S21)

        # plot update
        spec_list = np.append(spec_list, spec)
        plotter.update(xdata=spec_list, ydata=data)

    # save data
    exp_params.xdata = spec_list
    exp_params.data = data
    save_data(exp_params)

    # Post measurement
    plotter.fig.savefig(exp_params.full_file_path.replace('json','png'))
    logging.info('Measurement complete!')
    
    return exp_params

data = run_spec_1D(exp_params)

NameError: name 'base_path' is not defined

# Spec vs Power 2D

In [ ]:
# Set parameters 
exp_params = Dict({'exp_name': 'spec_vs_power_2D'
                    , 'CW_freq': 6e9                  
                    , 'power': -30
                    , 'num_points': 801
                    , 'avg': 2
                    , 'IF': 1e3
                    , 'power_start': -30
                    , 'power_end': 0
                    , 'power_step': 1
                    , 'spec_freq_start': 4e9
                    , 'spec_freq_end': 5e9
                    , 'spec_freq_step': 5e6
                    , 'spec_fixed_att' : 20
                    })
exp_params.update(exp_params_common)

def run_spec_vs_power_2D(exp_params):

    # file
    exp_params = create_full_file_name_and_path(exp_params)

    # plot initialization
    exp_params.xlabel = 'Flux1 (V)'
    exp_params.ylabel ='Power (dBm)'
    plotter = QuadPlotter(plot_dim=2
                        , xlabel=exp_params.xlabel
                        , ylabel=exp_params.ylabel
                        , title=exp_params.full_file_name
                        , figsize=(12,10))

    # initialize instruments
    pna.setupMeas('S21')
    pna.setSweepType('CW')
    pna.setCWFreq(exp_params.CW_freq)
    pna.setPwr(exp_params.power)
    pna.setIF(exp_params.IF)
    pna.avgCount(exp_params.avg)
    pna.numPoints(exp_params.num_points)

    # sweep
    power_sweep = np.arange(exp_params.power_start, exp_params.power_end + exp_params.power_step, exp_params.power_step)
    spec_sweep = np.arange(exp_params.spec_freq_start, exp_params.spec_freq_end + exp_params.spec_freq_step, exp_params.spec_freq_step)
    data = np.array([])
    for power in power_sweep:
        spec.setPwr(power)

        S21_list = np.array([])
        for spec in spec_sweep:
            spec.setFreq(spec)
            pna.avgClear()

            _, S21 = pna.getData(cplx=True) # S21 in linear scale
            S21_avg = np.mean(S21)
            S21_list = np.append(S21_list, S21_avg)        

        if data.size > 0:
            data = np.column_stack((data, S21_list))
        else:
            data = np.reshape(S21_list,(len(S21_list),1))

        # plot update
        plotter.update(x_axis=power_sweep, y_axis=spec_sweep, data=data)

    # save data
    exp_params.xdata = power_sweep
    exp_params.ydata = spec_sweep
    exp_params.data = data
    save_data(exp_params)

    # Post measurement
    plotter.fig.savefig(exp_params.full_file_path.replace('json','png'))
    logging.info('Measurement complete!')
    
    return exp_params

data = run_spec_vs_power_2D(exp_params)

# Spec vs flux, 2D

In [ ]:
# Set parameters 
exp_params = Dict({'exp_name': 'spec_vs_flux_2D'
                    , 'CW_freq': 6e9                  
                    , 'power': -30
                    , 'num_points': 801
                    , 'avg': 2
                    , 'IF': 1e3
                    , 'flux_start': -1
                    , 'flux_end': 1
                    , 'flux_step': 0.1
                    , 'spec_freq_start': 4e9
                    , 'spec_freq_end': 5e9
                    , 'spec_freq_step': 5e6
                    , 'spec_fixed_att' : 20
                    , 'cavity_retune': False
                    })
exp_params.update(exp_params_common)

def find_cavity_freq(exp_params):
      
    pna.setupMeas('S21')
    pna.setSweepType('linear')
    pna.setFreqStartEnd(exp_params.freq_start, exp_params.freq_end)
    pna.avgCount(exp_params.avg)
    pna.numPoints(exp_params.num_points)
    pna.setIF(exp_params.IF)
    pna.setPwr(exp_params.power)

    # sweep
    freq, S21 = pna.getData(cplx=True) # S21 in linear scale

    # find cavity bias freq
    (popt, pcov) = lorentzian_fit(freq, S21)
    
    return popt[1]

def lorentizina_ft(freq, S21):
    
    func = lambda x, a, f0, BW: a / (b + ((x-f0)/BW)**2)
    (popt, pcov) = curve_fit(func, freq, S21)
    
    return popt, pcov

def run_spec_vs_flux_2D(exp_params):
    # file
    exp_params = create_full_file_name_and_path(exp_params)

    # plot initialization
    exp_params.xlabel = 'Flux (V)'
    exp_params.ylabel = 'Spec. Frequency (Hz)'
    plotter = QuadPlotter(plot_dim=2
                        , xlabel=exp_params.xlabel
                        , ylabel=exp_params.ylabel
                        , title=exp_params.full_file_name
                        , figsize=(12,10))

    # initialize instruments
    pna.setupMeas('S21')
    pna.setSweepType('CW')
    pna.setCWFreq(exp_params.CW_freq)
    pna.setPwr(exp_params.power)
    pna.setIF(exp_params.IF)
    pna.avgCount(exp_params.avg)
    pna.numPoints(exp_params.num_points)

    # sweep
    flux_sweep = np.arange(exp_params.flux_start, exp_params.flux_end + exp_params.flux_step, exp_params.flux_step)
    spec_sweep = np.arange(exp_params.spec_freq_start, exp_params.spec_freq_end + exp_params.spec_freq_step, exp_params.spec_freq_step)
    data = np.array([])
    for flux in flux_sweep:
        flux.setVoltage(flux)

        if cavity_retune:
            exp_params_cavity_tune = Dict()
            exp_params_cavity_tune.freq_start = 6e9
            exp_params_cavity_tune.freq_end = 6.1e9
            exp_params_cavity_tune.power = -30
            exp_params_cavity_tune.num_points = 801
            exp_params_cavity_tune.avg = 2
            exp_params_cavity_tune.IF = 1e3

            new_cav_freq = find_cavity_freq(exp_params_cavity_tune)

            pna.setupMeas('S21')
            pna.setSweepType('CW')
            pna.setCWFreq(new_cav_freq)
            pna.avgCount(exp_params.avg)
            pna.numPoints(exp_params.num_points)
            pna.setPwr(exp_params.power)
            pna.setIF(exp_params.IF)

        S21_list = np.array([])
        for spec in spec_sweep:
            spec.setFreq(spec)
            pna.avgClear()

            _, S21 = pna.getData(cplx=True) # S21 in linear scale
            S21_avg = np.mean(S21)
            S21_list = np.append(S21_list, S21_avg)        

        if data.size > 0:
            data = np.column_stack((data, S21_list))
        else:
            data = np.reshape(S21_list,(len(S21_list),1))

        # plot update
        plotter.update(x_axis=flux_sweep, y_axis=spec_sweep, data=data)

    # save data
    exp_params.xdata = flux_sweep
    exp_params.ydata = spec_sweep
    exp_params.data = data
    save_data(exp_params)

    # Post measurement
    plotter.fig.savefig(exp_params.full_file_path.replace('json','png'))
    logging.info('Measurement complete!')
    
    return exp_params

data = run_spec_vs_flux_2D(exp_params)